### 困惑度、混淆度 (Perplexity, PPL)
在量化的過程中，會相當重視模型的損失程度，也就是說量化完之後的誤差有多大，這在語言模型裡面通常以**困惑度 (Perplexity, PPL)** 來表示，PPL 是用來描述模型預測下一個字詞的時候其不確定性如何。
* 因為語言模型每次推論時會產生一份機率表：
    * 如果這個機率表顯示「下個字是 X 的機率為 100% 」那就代表模型非常**明確**，這時困惑度就會相對較低。
    * 但如果模型覺得「嗯……好像每個字都有可能，那就代表模型相當**困惑**，這時困感度就會較高。
* 一般語言模型都是以**交叉熵 (Cross Entropy)** 當作**損失函數 (Loss Function)** 而困惑度就是對**損失值 (Loss)** 取**指數函數 (Exponential Function)** 的結果。

#### 如何計算困惑度？
1. 首先，要先來瞭解如何取得單次推論時的 Loss：
2. 接下來對 Loss 取指數函數就能得到困感度：

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

# model_dir = "meta-llama/Meta-Llama-3-8B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_dir,
#     device="auto",
# )

model = AutoModelForCausalLM.from_pretrained(...) 
tk = AutoTokenizer.from_pretrained(...)
input_ids = tk.encode(...)

with torch.no_grad():
    outputs = model.forward(input_ids=input_ids, labels=input_ids) 

print(outputs.loss)

ppl = torch.exp(outputs.loss)

#### 最簡單的評測方法
**固定序列長度**來計算困惑度，以 Wikitext 資料集為例：

##### 1. 首先透過分詞器進行分詞：

In [ ]:
from datasets import load_dataset

tk = AutoTokenizer.from_pretrained(...)

dataset = load_dataset(
    "wikitext",
    "wikitext-2-raw-vl",
    split="test",
)

Input_ids = list()
for item in dataset:
    text = item["text"] + "\n"
    tokens = tk.encode(text, add_special_tokens=False)
    input_ids.extend(tokens)

##### 2. 這裡先設定序列長度為 4096 來進行評估：
* 其中長度不足 4096 的最後一筆資料會被捨棄。

In [ ]:
import torch

seqlen = 4096
data_size = len(input_ids) // seqlen # 計算序列數量
input_ids= input_ids[: data_size * seqlen] # 捨棄最後一筆

##### 3. 然後將 `input_ids` 轉入 `Tensor`，並在每個序列的開頭加上 BOS Token，最後把 `Tensor` 移動到對應的裝置上：
* 雖然這裡長度設定為 4096，但是因為加上額外的一個 BOS Token，所以填型的實際推論長度是 4097，不過計算 Loss 時會位移一格，因此最終得到的困惑度依然是以序列長度 4096 算出來的。

In [ ]:
input_ids = torch.LongTensor(input_ids).view(data_size, seqlen)
bos_token = torch.full(
    (data_size, 1),
    tk.bos_token_id,
    dtype=torch.int64,
)

input_ids = torch.concat((bos_token, input_ids), dim=1)
input_ids = input_ids.to(model.device)

##### 4. 接下來開始對測試資料進行推論：
* `n11s` 代表 Negative Log-Likelihood，與 Cross Entropy 基本上是等價的概念。

In [ ]:
nlls = list()
for i in range(data_size):
    batch = input_ids[i : i+1]
    with torch.no_grad():
        outputs = model.forward(batch, labels=batch)
    nlls.append(outputs.loss)
ppl = torch.exp(torch.stack(nlls).mean())
print(ppl)

##### 5. 因為評估的過程可能會花上一段時間，所以可以借助 `tqdm` 套件來顯示進度：

In [ ]:
from tqdm import trange

batch_size = 1
nlls = list()
with trange(0, data_size, batch_size) as prog:
    for i in prog:
        batch = input_ids[i : i + batch_size]
        outputs = model.forward(batch, labels=batch)
        nlls.append(outputs.loss)
        ppl = torch.exp(torch.stack(nlls).mean())
        prog.desc= f"ppl: {ppl:.4f}"

#### 結論
* 這裡拿 Llama 3 8B Instruct 簡單比較一下 Float16、BFloat16 與 BitsAndBytes 之間困惑度的差異：
    | 類型      | 困惑度 |
    | --------- | ------ |
    | Float16   | 7.7494 |
    | BFloat16  | 7.7570 |
    | BNB 8-Bit | 7.7730 |
    | BNB 4-Bit | 8.1899 |
    * 可以看到 FP16 與 BF16 幾乎沒有差異，BNB 8-Bit 也與 FP16 十分相近，但 BNB 4-Bit 就稍微有點距離了，不過這個差距還算可以接受。
    * 根據筆者的經驗， 通常困惑度的差距到達 1~2 點左右就會開始感受到實際輸出品質在下降了。
* 困惑度並不是個絕對性的指標，在同個模型內做比較是滿有參考性的，但如果是不同模型或者不同訓練資料時，用困惑度進行互相比較就需要更加謹慎一些。
    * 例如分詞器的不同，可能會造成同一份文本對不同模型而言有不同的長度，在困惑度的比較上也會因此產生差異。
* 在合理的量化下，模型的參數量通常會比模型的大小來的重要。
    * 就算用 16-Bit 去跑一個 7B 的模型，其效果也不會比 4-Bit 的 13B 模型還要好，即使 7B 16-Bit 的模型在賣際大小上比 13B 4-Bit 還要大的多。
    * 因此模型的能力主要取決於參數量的多寡，而資料型態僅影響運算的精準度而已。
* 即便模型變小了，但模型的總計算量原則上是不變的，因此並不會因為模型從 16-Bit 變成 8-Bit 後，推論速度就快上兩倍。
* 真正有影響的是每個權重的位元數縮小後，在運算上可以減少傳輸頻寬，使**吞吐量 (Throughput)** 提高，也就是每個**批次 (Batch)** 能容納的 Token 變多了。
* 或者從軟硬體上針對特殊設計的資料型態進行加速運算，但是基本上計算量還是一樣的。